In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

# nltk is the Natural Language Toolkit. It is a set of libaries to aid with natural language processing
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, opinion_lexicon
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
nltk.download('omw-1.4') # Open Multilingual Wordnet

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# Sentence tokenizer. A tokenizer divides text into a list of sentences.
# This module is smart enough to distinguish between periods that demark sentences and
# other periods, such as in Mr. Smith. It also recognizes that sentences don't always start with capital letters.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Stopwords module. Stopwords are common words such as "the" and "and" that convey little information
# and often should be stripped out of text before machine learning is performed.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.download('wordnet') # WordNet module.

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# nltk's opinion lexicon is a classification of positive and negative words.
# It is a built-in tool to help with sentiment analysis.
nltk.download('opinion_lexicon')

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [7]:
# The dataset is a collection of comments from a Facebook depression and anxiety group.
# Our goal is to classify them as positive and negative sentiment.
data=pd.read_excel('sentiment20230525/Depression & Anxiety Facebook page Comments Text.xlsx')

In [8]:
# We prepare a clean set of comments with two transformations.
# 1) Stopwords are removed, as they convey little useful information about sentiment
# 2) Words are lemmatized, which means they are converted to base forms.
# Lemmatization is different from stemming and typically more intelligent and more considerate of context.

stopword = stopwords.words("english")
lem=WordNetLemmatizer()

def remove(x):
    # Strip out numbers and punctuation, regularize the spacing.
    # The following does not affect capitalization.
    x=re.sub("[^A-z" "]+"," ",x)
    s=""
    for i in x.split():
        if i not in stopword:
            i=lem.lemmatize(word=i)
            s=s+" "+i
    return s

data["Comments_clean"] = data["Comments Text"].apply(remove)

In [9]:
# Some demonstration of nltk's lemmatizer

print("cats:",lem.lemmatize(word="cats"))
print("Cats:",lem.lemmatize(word="Cats")) # Capital letters trip up the lemmatizer
print("caring:",lem.lemmatize(word="caring")) # By default, it handles nouns.
print("caring (as a verb):",lem.lemmatize("caring","v"))

cats: cat
Cats: Cats
caring: caring
caring (as a verb): care


In [10]:
# An example of a comment after processing.
data["Comments_clean"][0]

' So ask two illness similar tend moodiness involved impulsivity self damaging behavior compensatory measure Otherwise really similar seem like one another often show person making life chaotic difficult dysfunctional intense So friend concerned may one illness ashamed It definitely fault They instead get help professional INSIST treated appropriately correct type therapy medication'

In [11]:
# Valent Aware Dictionary and sEntiment Reasoner
# See https://github.com/cjhutto/vaderSentiment
#pip install vaderSentiment

In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [13]:
vs = SentimentIntensityAnalyzer()

In [14]:
# VADER can process emojis.
vs.emojis['😀']

'grinning face'

In [15]:
# VADER is an out-of-the-box sentiment classifier trained specifically on social media.
# Here is an example.
vs.polarity_scores(data["Comments_clean"][3])

{'neg': 0.309, 'neu': 0.586, 'pos': 0.105, 'compound': -0.9905}

In [16]:
# Here is another example.
d=vs.polarity_scores(data["Comments_clean"][0])
d.pop("compound")
d

{'neg': 0.303, 'neu': 0.517, 'pos': 0.18}

In [17]:
# 'compound' is the positive minus the negative values. The bigger it is, the more positive VADER assesses the sentence.
def polarity(sent):
    d=vs.polarity_scores(sent)
    if d["compound"]>0.05:
        return "positive"
    elif d["compound"]<-0.05:
        return "negative"
    else:
        return "neutral"


In [18]:
data["Target"] = data["Comments_clean"].apply(polarity)
data.head()

Comments Text  \
0  So, when you ask what the two illnesses are......   
1  In addition, people with BPD quite often have ...   
2  Borderline Personality Disorder, like all othe...   
3  LONG ANSWER: Bipolar disease is caused by a ch...   
4  Well think of bipolar as a rollar coaster you ...   

                                      Comments_clean    Target  
0   So ask two illness similar tend moodiness inv...  negative  
1   In addition people BPD quite often weak self ...  negative  
2   Borderline Personality Disorder like personal...  negative  
3   LONG ANSWER Bipolar disease caused chemical i...  negative  
4   Well think bipolar rollar coaster high low pe...  negative

In [19]:
# As one might guess, negative statements outweigh the positive. A minority are classified as neutral.
data.Target.value_counts()

negative    4038
positive    2925
neutral      182
Name: Target, dtype: int64

In [20]:
# After doing some preprocessing, save the data as an intermediate file.
data.to_csv("New_7k_clean_data.csv",index=False)

# Machine Learning

In [21]:
data_new = pd.read_csv("New_7k_clean_data.csv")
data_new.head()

Comments Text  \
0  So, when you ask what the two illnesses are......   
1  In addition, people with BPD quite often have ...   
2  Borderline Personality Disorder, like all othe...   
3  LONG ANSWER: Bipolar disease is caused by a ch...   
4  Well think of bipolar as a rollar coaster you ...   

                                      Comments_clean    Target  
0   So ask two illness similar tend moodiness inv...  negative  
1   In addition people BPD quite often weak self ...  negative  
2   Borderline Personality Disorder like personal...  negative  
3   LONG ANSWER Bipolar disease caused chemical i...  negative  
4   Well think bipolar rollar coaster high low pe...  negative

In [22]:
# Based on our classification, few of the posts are neutral.
sns.countplot(x=data_new.Target, data=data_new)

<Axes: xlabel='Target', ylabel='count'>

In [23]:
data_new.drop(columns="Comments Text",inplace=True)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
# The count vectorizer converts sentences into arrays, counting the number of times each word appears in a corpus.
# The 'lowercase' option coverts all words to lower case before vectorizing.
cv = CountVectorizer(lowercase=True)

In [26]:
# There are 13211 words in the corpus
data_array = cv.fit_transform(data_new.Comments_clean).toarray()
data_array.shape

(7145, 13211)

In [27]:
# Due to the large number of distinct words in the corpus, comments are portrayed by sparse vectors.
data_array[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [28]:
import numpy as np

In [29]:
# This potrays unique word counts in the vectors.
# For example, in the first comment (vectorized as data_array[0]), there are words that appear 0 times,
# words that appear 1 time, and words that appear 2 times.
for i in range(10):
    print(np.unique(data_array[i]))

[0 1 2]
[0 1 2 3 4 5 6 7]
[ 0  1  2  3  4  5  6  7  8 11]
[0 1 2 3 4]
[0 1 2]
[0 1]
[0 1 2 4]
[0 1]
[0 1 2 3 4 5]
[0 1 2 3]


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [31]:
# Train test split for model training and evaluation. 10% of the examples will be in the test set.
# The random_state ensures that the split will be the same on every run.
x, x_test, y, y_test = train_test_split(data_array, data_new.Target, random_state=42, test_size=0.1)

In [32]:
[x.shape, x_test.shape]

[(6430, 13211), (715, 13211)]

In [33]:
rf = RandomForestClassifier()

In [34]:
rf.fit(x,y)

RandomForestClassifier()

In [35]:
pred=rf.predict(x_test)

In [36]:
# This shows ~77% accuracy for the random forest classifier to predict the sentiment label
# that was assigned by VADER.
accuracy_score(y_test, pred)*100

76.92307692307693

In [37]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.80      0.83      0.81       407
     neutral       0.67      0.10      0.17        21
    positive       0.73      0.73      0.73       287

    accuracy                           0.77       715
   macro avg       0.73      0.55      0.57       715
weighted avg       0.77      0.77      0.76       715



In [38]:
# As might be expected, the classifier has poor recall, though decent precision, with neutral comments.
confusion_matrix(y_test, pred)

array([[339,   0,  68],
       [  9,   2,  10],
       [ 77,   1, 209]], dtype=int64)